<a href="https://colab.research.google.com/github/currencyfxjle/An-lisis-Causal-y-Predictivo-Utilizando-Regresi-n/blob/main/Knapsack_GDF_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 2.9 MB/s eta 0:00:00


# **Full Project Allocation / Bank Credit**

In [29]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# -------------------------------------------------------------------------
# 0. Authenticate and open the Google Sheet
# -------------------------------------------------------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

title = "2025 Project Agenda"
sheet = gc.open(title)

# -------------------------------------------------------------------------
# 1. Fetch Supply
# -------------------------------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Keep only rows marked 'Y'
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']

    # Convert numeric fields
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(supply_df['Project_date'], errors='coerce')

    # Convert GDF Profit to numeric (e.g., "25%" -> 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )

    # Drop rows missing critical columns
    supply_df = supply_df.dropna(subset=['Est Cash Amount','Project_date','GDF Profit'])

    # Ensure we have a Project ID
    if 'Project ID' not in supply_df.columns or supply_df['Project ID'].isnull().all():
        supply_df['Project ID'] = supply_df['Project Name']
    else:
        supply_df['Project ID'] = supply_df['Project ID'].fillna(supply_df['Project Name'])

    # If needed, derive "Project Category" from chunk size
    if 'Project Category' not in supply_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        supply_df['Project Category'] = pd.cut(
            supply_df['Est Cash Amount'], bins=bins, labels=labels
        )

    # Rename "State" -> "Project State" if needed
    if 'State' in supply_df.columns:
        supply_df.rename(columns={'State':'Project State'}, inplace=True)

    # Ensure Priority is numeric
    if 'Priority' not in supply_df.columns:
        raise ValueError("Supply data must contain a 'Priority' column.")

    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])

    # Sort supply similarly to your original logic
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Keep an original copy of Est Cash Amount
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']

    # Add 'Fully Allocated' flag
    supply_df['Fully Allocated'] = False

    return supply_df

# -------------------------------------------------------------------------
# 2. Fetch Demand
# -------------------------------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename columns if needed
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Convert numeric fields
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',':''}, regex=True)
    )

    # Drop 'pending_amount' if it exists
    if 'pending_amount' in demand_df.columns:
        demand_df.drop(columns=['pending_amount'], inplace=True)

    # Convert dates
    demand_df['Purchaser_Date'] = pd.to_datetime(demand_df['Purchaser_Date'], errors='coerce')
    demand_df = demand_df.dropna(subset=['approximated_appetite','Purchaser_Date'])

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Rename "State" -> "Purchaser State" if needed
    if 'State' in demand_df.columns:
        demand_df.rename(columns={'State':'Purchaser State'}, inplace=True)

    # Ensure "Purchaser Category"
    if 'Purchaser Category' not in demand_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        demand_df['Purchaser Category'] = pd.cut(
            demand_df['approximated_appetite'], bins=bins, labels=labels
        )

    # State Priority: CA=1, others=2
    demand_df['State Priority'] = demand_df['Purchaser State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # Sort demand by appetite desc, then state priority asc
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# -------------------------------------------------------------------------
# 3. Allocate function (with optional bank bridging)
# -------------------------------------------------------------------------
def allocate_chunks_no_partial(
    supply_df: pd.DataFrame,
    demand_df: pd.DataFrame,
    bank_credit_limit: float = 20_000_000
):
    """
    Allocates supply to demand. If bank_credit_limit=0 => no bridging,
    matching the non-financing approach. If >0 => attempt bridging.

    New bridging logic:
    - Direct buyer is still chosen by largest appetite + CA-first (as before).
    - If no direct buyer is found, use Bank bridging.
    - For bridging, pick the earliest Purchaser Date >= project date (ignoring appetite size as a priority).
    - After bridging occurs, re-sort the local demand back to the original (appetite desc, CA-first) ordering
      so that future direct allocations still favor highest appetite + CA-first.
    """

    supply_df = supply_df.copy()
    demand_df = demand_df.copy()

    # Convert to record-lists so we can manipulate easily
    supply_records = supply_df.to_dict('records')
    demand_records = demand_df.to_dict('records')

    allocations = []
    financed_chunks = []
    bank_available = bank_credit_limit

    allocated_projects = set()

    # 1) Find a direct purchaser => largest appetite + CA-first
    def find_direct_purchaser(project_date, chunk_cost, d_records):
        eligible = [
            (p, i) for i, p in enumerate(d_records)
            if p['Purchaser_Date'] <= project_date and p['remaining_appetite'] >= chunk_cost
        ]
        if not eligible:
            return None
        # Sort by (State Priority asc, remaining_appetite desc)
        eligible.sort(key=lambda x: (x[0]['State Priority'], -x[0]['remaining_appetite']))
        return eligible[0]

    # 2) Find a future purchaser for bridging => earliest date >= project date
    #    ignoring appetite-based priority, beyond the requirement that they
    #    have enough appetite in the first place.
    def find_future_purchaser_earliest_date(project_date, chunk_cost, d_records):
        future = [
            (p, i) for i, p in enumerate(d_records)
            if p['Purchaser_Date'] >= project_date and p['remaining_appetite'] >= chunk_cost
        ]
        if not future:
            return None
        # Sort solely by earliest Purchaser_Date (ascending).
        # If there's a tie in date, you could break ties by state or something else,
        # but we keep it simple per your requirement.
        future.sort(key=lambda x: x[0]['Purchaser_Date'])
        return future[0]

    # Helper to re-sort demand back to the original logic:
    def resort_demand_to_original(d_records):
        # Original logic: appetite desc, then state priority asc
        d_records.sort(key=lambda x: ( -x['approximated_appetite'], x['State Priority'] ))
        return d_records

    # Group supply by Project Name in the same order they appear
    grouped_supply = supply_df.groupby('Project Name', sort=False)

    # ---------------------------------------------------------------------
    # Allocation loop
    # ---------------------------------------------------------------------
    for project_name, group in grouped_supply:
        # If the entire project is already allocated, skip
        if group['Fully Allocated'].all():
            continue

        project_ok = True
        project_chunks = group.to_dict('records')

        # We'll store allocations temporarily, to confirm we can do the entire project
        temp_allocs = []
        temp_financed = []

        # Make a local copy of the demand + bank, so if we fail we can roll back
        local_demand = [dict(d) for d in demand_records]
        local_bank = bank_available

        # Sort chunks internally (as you did originally):
        # by project priority asc, then chunk cost desc
        sorted_chunks = sorted(
            project_chunks,
            key=lambda x: (x['Priority'], -x['Est Cash Amount'])
        )

        # ---------------------------------------------------------------------
        # For each chunk
        # ---------------------------------------------------------------------
        for chunk in sorted_chunks:
            if chunk.get('Fully Allocated'):
                continue

            chunk_cost = chunk['Est Cash Amount']
            p_name = chunk['Project Name']
            p_cat  = chunk.get('Project Category','')
            p_st   = chunk.get('Project State','')
            p_gdf  = chunk['GDF Profit']
            p_id   = chunk['Project ID']
            p_priority = chunk['Priority']
            p_chunk_date = chunk['Project_date']

            # 1) Attempt direct
            direct = find_direct_purchaser(p_chunk_date, chunk_cost, local_demand)
            if direct:
                pur, pur_idx = direct
                pur['remaining_appetite'] -= chunk_cost

                temp_allocs.append({
                    "Purchaser Name": pur['Purchaser Name'],
                    "Purchaser State": pur.get('Purchaser State',''),
                    "Purchaser Date": pur['Purchaser_Date'],
                    "Purchaser Category": pur.get('Purchaser Category',''),

                    "Project Name": p_name,
                    "Project Date": p_chunk_date,
                    "Project Category": p_cat,
                    "Project State": p_st,
                    "GDF Profit": p_gdf,
                    "Priority": p_priority,

                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": pur['remaining_appetite'],
                    "Original Appetite": pur['approximated_appetite'],

                    "Project ID": p_id,
                    "Result": True
                })
                # Continue to next chunk
                continue

            # 2) No direct buyer => Attempt Bank bridging
            if local_bank >= chunk_cost and bank_credit_limit > 0:
                future = find_future_purchaser_earliest_date(p_chunk_date, chunk_cost, local_demand)
                if future:
                    f_pur, f_idx = future

                    # Bank "entry"
                    local_bank -= chunk_cost
                    temp_allocs.append({
                        "Purchaser Name": "Bank Credit",
                        "Purchaser State": "N/A",
                        "Purchaser Date": p_chunk_date,
                        "Purchaser Category": "N/A",

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": chunk_cost,
                        "Remaining Appetite After": 0,
                        "Original Appetite": 0,

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Bank "exit" to earliest-date future purchaser
                    f_pur['remaining_appetite'] -= chunk_cost
                    exit_date = f_pur['Purchaser_Date']
                    local_bank += chunk_cost

                    temp_allocs.append({
                        "Purchaser Name": f_pur['Purchaser Name'],
                        "Purchaser State": f_pur.get('Purchaser State',''),
                        "Purchaser Date": exit_date,
                        "Purchaser Category": f_pur.get('Purchaser Category',''),

                        "Project Name": p_name,
                        "Project Date": p_chunk_date,
                        "Project Category": p_cat,
                        "Project State": p_st,
                        "GDF Profit": p_gdf,
                        "Priority": p_priority,

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": -chunk_cost,  # negative indicates exit from bank
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],

                        "Project ID": p_id,
                        "Result": True
                    })

                    # Record bridging info
                    temp_financed.append({
                        "Project ID": p_id,
                        "Project Name": p_name,
                        "Chunk Cost": chunk_cost,
                        "Purchase Date": p_chunk_date,
                        "Exit Date": exit_date,
                        "Exit Purchaser Name": f_pur['Purchaser Name'],
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],
                        "GDF Profit": p_gdf
                    })

                    # **Re-sort** local_demand back to the original
                    # (largest appetite desc, then CA=1).
                    local_demand = resort_demand_to_original(local_demand)

                else:
                    # Bridging not possible => fail project
                    project_ok = False
                    break
            else:
                # No direct buyer & bridging not possible => fail project
                project_ok = False
                break

        # ---------------------------------------------------------------------
        # Decide if the project was fully allocated
        # ---------------------------------------------------------------------
        if project_ok:
            # Mark as allocated
            allocated_projects.add(project_name)

            # Commit local changes
            demand_records = local_demand
            bank_available = local_bank

            # Add to final allocations
            allocations.extend(temp_allocs)
            financed_chunks.extend(temp_financed)

            # Mark supply "Fully Allocated"
            supply_df.loc[group.index, 'Fully Allocated'] = True
        else:
            # Project not fully allocated => revert
            for ch in group.to_dict('records'):
                allocations.append({
                    "Purchaser Name": None,
                    "Purchaser State": None,
                    "Purchaser Date": None,
                    "Purchaser Category": None,

                    "Project Name": ch['Project Name'],
                    "Project Date": ch['Project_date'],
                    "Project Category": ch.get('Project Category',''),
                    "Project State": ch.get('Project State',''),
                    "GDF Profit": ch['GDF Profit'],
                    "Priority": ch['Priority'],

                    "Allocated Amount": 0.0,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": None,
                    "Original Appetite": None,

                    "Project ID": ch['Project ID'],
                    "Result": False
                })

    # ---------------------------------------------------------------------
    # Build Allocations DataFrame
    # ---------------------------------------------------------------------
    allocations_df = pd.DataFrame(allocations)
    alloc_cols = [
        "Purchaser Name","Purchaser State","Purchaser Date","Purchaser Category",
        "Project Name","Project Date","Project Category","Project State","GDF Profit",
        "Priority","Allocated Amount","Financed Amount","Remaining Appetite After",
        "Original Appetite","Project ID","Result"
    ]
    allocations_df = allocations_df[[c for c in alloc_cols if c in allocations_df.columns]]

    # ---------------------------------------------------------------------
    # Build Financing DataFrame
    # ---------------------------------------------------------------------
    financed_df = pd.DataFrame(financed_chunks)
    fin_cols = [
        "Project ID","Project Name","Chunk Cost","Purchase Date","Exit Date",
        "Exit Purchaser Name","Remaining Appetite After","Original Appetite",
        "GDF Profit"
    ]
    for cc in fin_cols:
        if cc not in financed_df.columns:
            financed_df[cc] = None
    financed_df = financed_df[fin_cols]

    # If bridging was used, calculate interest & profit
    if not financed_df.empty:
        financed_df['Purchase Date'] = pd.to_datetime(financed_df['Purchase Date'], errors='coerce')
        financed_df['Exit Date'] = pd.to_datetime(financed_df['Exit Date'], errors='coerce')

        # Months of bank credit
        financed_df['Time of Bank Credit'] = financed_df.apply(
            lambda row: (
                (row['Exit Date'].year - row['Purchase Date'].year) * 12
                + (row['Exit Date'].month - row['Purchase Date'].month)
            ),
            axis=1
        )

        # 1% monthly interest
        financed_df['Cost of Interest'] = financed_df['Chunk Cost'] * 0.01 * financed_df['Time of Bank Credit']

        financed_df['Gross Profit'] = financed_df['Chunk Cost'] * financed_df['GDF Profit']
        financed_df['Net Profit']   = financed_df['Gross Profit'] - financed_df['Cost of Interest']
        financed_df['Net profit %'] = financed_df['Net Profit'] / financed_df['Chunk Cost']
    else:
        financed_df['Time of Bank Credit'] = None
        financed_df['Cost of Interest'] = None
        financed_df['Gross Profit'] = None
        financed_df['Net Profit'] = None
        financed_df['Net profit %'] = None

    # ---------------------------------------------------------------------
    # Summary Metrics
    # ---------------------------------------------------------------------
    # Total allocated (exclude the temporary "Bank Credit" allocations)
    total_alloc = (
        allocations_df['Allocated Amount'].sum()
        - allocations_df.loc[allocations_df['Purchaser Name'] == 'Bank Credit', 'Allocated Amount'].sum()
    )
    financed_used = allocations_df.loc[allocations_df['Financed Amount'] > 0, 'Financed Amount'].sum()
    bank_remaining = bank_available

    # Demand side
    original_total_appetite = demand_df['approximated_appetite'].sum()
    demand_records_df = pd.DataFrame(demand_records)
    remaining_demand = demand_records_df['remaining_appetite'].sum()

    # Supply side
    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    unallocated_projects = set(supply_df['Project Name'].unique()) - allocated_projects
    unallocated_supply_df = supply_df[supply_df['Project Name'].isin(unallocated_projects)]
    remaining_supply_value = unallocated_supply_df['Est Cash Amount'].sum()

    pct_alloc = 0.0
    if original_total_project_value > 0:
        pct_alloc = (total_alloc / original_total_project_value) * 100

    # Core summary
    summary_data = [
        {"Metric": "--- RESULTS SUMMARY ---", "Value": ""},
        {"Metric": "Total Allocated", "Value": f"{total_alloc:,.2f}"},
        {"Metric": "Financed (Bank Used)", "Value": f"{financed_used:,.2f}"},
        {"Metric": "Bank Credit Remaining", "Value": f"{bank_remaining:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "DEMAND SUM", "Value": ""},
        {"Metric": "Original Total Appetite", "Value": f"{original_total_appetite:,.2f}"},
        {"Metric": "Remaining Demand", "Value": f"{remaining_demand:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": f"{pct_alloc:.2f}%"},

        {"Metric": "", "Value": ""},
        {"Metric": "SUPPLY SUM", "Value": ""},
        {"Metric": "Original Total Project Value", "Value": f"{original_total_project_value:,.2f}"},
        {"Metric": "Remaining Supply", "Value": f"{remaining_supply_value:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "--- GROSS PROFIT METRICS ---", "Value": ""},
        {"Metric": "Total Gross Profit (all projects)", "Value": ""},
        {"Metric": "Total Gross Profit Allocated", "Value": ""},
        {"Metric": "Total Gross Profit Unallocated", "Value": ""}
    ]
    summary_df = pd.DataFrame(summary_data)

    # ---------------------------------------------------------------------
    # Supply Summary with Gross Profit
    # ---------------------------------------------------------------------
    project_results = allocations_df.groupby('Project Name')['Result'].all().reset_index()
    project_results.rename(columns={'Result': 'Result'}, inplace=True)
    supply_after_df = supply_df.merge(project_results, on='Project Name', how='left')
    supply_after_df['Result'] = supply_after_df['Result'].fillna(False)

    supply_summary = supply_after_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Chunks_After=('Result', lambda x: x.sum()),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),
        GDF_Profit=('GDF Profit', 'max'),
        Priority=('Priority', 'max')
    ).reset_index()

    supply_summary['Gross_Profit'] = (
        supply_summary['Total_Project_Value'] * supply_summary['GDF_Profit']
    )

    total_gross_profit = supply_summary['Gross_Profit'].sum()
    total_gross_profit_allocated = supply_summary.loc[supply_summary['Total_Chunks_After'] > 0, 'Gross_Profit'].sum()
    total_gross_profit_unallocated = supply_summary.loc[supply_summary['Total_Chunks_After'] == 0, 'Gross_Profit'].sum()

    summary_df.iloc[-3, summary_df.columns.get_loc("Value")] = f"{total_gross_profit:,.2f}"
    summary_df.iloc[-2, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_allocated:,.2f}"
    summary_df.iloc[-1, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_unallocated:,.2f}"

    # ---------------------------------------------------------------------
    # Demand After
    # ---------------------------------------------------------------------
    updated_demand_df = demand_records_df.copy()
    updated_demand_df['allocated_amount'] = (
        updated_demand_df['approximated_appetite'] - updated_demand_df['remaining_appetite']
    )
    updated_demand_df["Financing"] = 0.0

    # If bridging was used, let's track how much each purchaser ended up financing
    if not allocations_df.empty and "Financed Amount" in allocations_df.columns:
        financing_by_purchaser = (
            allocations_df[allocations_df['Financed Amount'] < 0]
            .groupby('Purchaser Name')['Financed Amount'].sum()
            .abs()
            .reset_index(name='Financing')
        )
        updated_demand_df = updated_demand_df.merge(
            financing_by_purchaser, on='Purchaser Name', how='left', suffixes=('', '_calc')
        )
        updated_demand_df['Financing'] = updated_demand_df['Financing_calc'].fillna(0)
        updated_demand_df.drop(columns=['Financing_calc'], inplace=True)

    demand_after_cols = [
        "Purchaser Name",
        "approximated_appetite",
        "allocated_amount",
        "Purchaser State",
        "Purchaser_Date",
        "Financing",
        "remaining_appetite",
        "Purchaser Category",
        "State Priority"
    ]
    for c in demand_after_cols:
        if c not in updated_demand_df.columns:
            updated_demand_df[c] = None

    updated_demand_df = updated_demand_df[demand_after_cols]

    return (
        allocations_df,      # 1) Detailed chunk-by-chunk allocations
        financed_df,         # 2) Bank bridging details
        summary_df,          # 3) High-level summary
        supply_summary,      # 4) Per-project supply summary
        supply_after_df,     # 5) Supply with allocation results
        updated_demand_df    # 6) Demand after final allocations
    )

# -------------------------------------------------------------------------
# 4. Main Execution
# -------------------------------------------------------------------------
def main():
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        updated_supply,
        updated_demand_df
    ) = allocate_chunks_no_partial(supply_df, demand_df, bank_credit_limit=20_000_000)

    # Print the summary to console
    print(summary_df.to_string(index=False))

    # Save to Excel in multiple tabs
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)
        financed_df.to_excel(writer, sheet_name="Financing Chunks", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)
        updated_supply.to_excel(writer, sheet_name="SupplyAfter", index=False)
        updated_demand_df.to_excel(writer, sheet_name="DemandAfter", index=False)

    print("\nResults saved to 'allocation_results.xlsx' with:")
    print(" - Allocations (all direct + bank bridging if applicable)")
    print(" - Financing Chunks (details of bridging, interest & profit columns)")
    print(" - Summary (result metrics including Gross Profit)")
    print(" - Supply Summary (detailed project allocations and Gross Profit)")
    print(" - SupplyAfter (with 'Result' column indicating allocation success)")
    print(" - DemandAfter (final states of demand)")

if __name__ == "__main__":
    main()

                           Metric         Value
          --- RESULTS SUMMARY ---              
                  Total Allocated  7,349,366.00
             Financed (Bank Used)  1,773,816.00
            Bank Credit Remaining 20,000,000.00
                                               
                       DEMAND SUM              
          Original Total Appetite 10,957,745.00
                 Remaining Demand  3,608,379.00
                                               
            % of Allocated Supply       100.00%
                                               
                       SUPPLY SUM              
     Original Total Project Value  7,349,366.00
                 Remaining Supply          0.00
                                               
     --- GROSS PROFIT METRICS ---              
Total Gross Profit (all projects)  1,531,247.23
     Total Gross Profit Allocated  1,531,247.23
   Total Gross Profit Unallocated          0.00

Results saved to 'allocation_results.xl

In [58]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# -------------------------------------------------------------------------
# 0. Authenticate and open the Google Sheet
# -------------------------------------------------------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

title = "2025 Project Agenda"
sheet = gc.open(title)

# -------------------------------------------------------------------------
# 1. Fetch Supply
# -------------------------------------------------------------------------
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Keep only rows marked 'Y'
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']

    # Convert numeric fields
    supply_df['Est Cash Amount'] = pd.to_numeric(
        supply_df['Est Cash Amount'].replace({',': ''}, regex=True)
    )
    supply_df['Project_date'] = pd.to_datetime(supply_df['Project_date'], errors='coerce')

    # Convert GDF Profit to numeric (e.g., "25%" -> 0.25)
    supply_df['GDF Profit'] = (
        supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    )

    # Drop rows missing critical columns
    supply_df = supply_df.dropna(subset=['Est Cash Amount','Project_date','GDF Profit'])

    # Ensure we have a Project ID
    if 'Project ID' not in supply_df.columns or supply_df['Project ID'].isnull().all():
        supply_df['Project ID'] = supply_df['Project Name']
    else:
        supply_df['Project ID'] = supply_df['Project ID'].fillna(supply_df['Project Name'])

    # If needed, derive "Project Category" from chunk size
    if 'Project Category' not in supply_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        supply_df['Project Category'] = pd.cut(
            supply_df['Est Cash Amount'], bins=bins, labels=labels
        )

    # Rename "State" -> "Project State" if needed
    if 'State' in supply_df.columns:
        supply_df.rename(columns={'State':'Project State'}, inplace=True)

    # Ensure Priority is numeric
    if 'Priority' not in supply_df.columns:
        raise ValueError("Supply data must contain a 'Priority' column.")

    supply_df['Priority'] = pd.to_numeric(supply_df['Priority'], errors='coerce')
    supply_df = supply_df.dropna(subset=['Priority'])

    # Sort supply similarly to your original logic
    supply_df = supply_df.sort_values(
        by=['Priority', 'GDF Profit', 'Est Cash Amount', 'Project_date'],
        ascending=[True, False, False, True]
    )

    # Keep an original copy of Est Cash Amount
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']

    # Add 'Fully Allocated' flag
    supply_df['Fully Allocated'] = False

    return supply_df

# -------------------------------------------------------------------------
# 2. Fetch Demand
# -------------------------------------------------------------------------
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Rename columns if needed
    demand_df = demand_df.rename(columns={'purchaser_name': 'Purchaser Name'})

    # Convert numeric fields
    demand_df['approximated_appetite'] = pd.to_numeric(
        demand_df['approximated_appetite'].replace({',':''}, regex=True)
    )

    # Drop 'pending_amount' if it exists
    if 'pending_amount' in demand_df.columns:
        demand_df.drop(columns=['pending_amount'], inplace=True)

    # Convert dates
    demand_df['Purchaser_Date'] = pd.to_datetime(demand_df['Purchaser_Date'], errors='coerce')
    demand_df = demand_df.dropna(subset=['approximated_appetite','Purchaser_Date'])

    # Track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Rename "State" -> "Purchaser State" if needed
    if 'State' in demand_df.columns:
        demand_df.rename(columns={'State':'Purchaser State'}, inplace=True)

    # Ensure "Purchaser Category"
    if 'Purchaser Category' not in demand_df.columns:
        bins = [0,350000,500000,800000,1000000,float('inf')]
        labels= ['0-350k','350k-500k','500k-800k','800k-1M','1M+']
        demand_df['Purchaser Category'] = pd.cut(
            demand_df['approximated_appetite'], bins=bins, labels=labels
        )

    # State Priority: CA=1, others=2
    demand_df['State Priority'] = demand_df['Purchaser State'].apply(
        lambda x: 1 if x == 'CA' else 2
    )

    # Sort demand by appetite desc, then state priority asc
    demand_df = demand_df.sort_values(
        by=['approximated_appetite', 'State Priority'],
        ascending=[False, True]
    )

    return demand_df

# -------------------------------------------------------------------------
# 3. Allocate function (with optional bank bridging)
# -------------------------------------------------------------------------
def allocate_chunks_no_partial(
    supply_df: pd.DataFrame,
    demand_df: pd.DataFrame,
    bank_credit_limit: float = 20_000_000
):
    """
    Allocates supply to demand. If bank_credit_limit=0 => no bridging,
    matching the non-financing approach. If >0 => attempt bridging.
    """

    supply_df = supply_df.copy()
    demand_df = demand_df.copy()

    demand_records = demand_df.to_dict('records')
    bank_available = bank_credit_limit

    allocations = []
    financed_chunks = []
    allocated_projects = set()

    # 1) Find a direct purchaser => largest appetite + CA-first
    def find_direct_purchaser(project_date, chunk_cost, d_records):
        eligible = [
            (p, i) for i, p in enumerate(d_records)
            if p['Purchaser_Date'] <= project_date and p['remaining_appetite'] >= chunk_cost
        ]
        if not eligible:
            return None
        # Sort by (State Priority asc, remaining_appetite desc)
        eligible.sort(key=lambda x: (x[0]['State Priority'], -x[0]['remaining_appetite']))
        return eligible[0]

    # 2) Find a future purchaser for bridging => earliest date >= project date
    def find_future_purchaser_earliest_date(project_date, chunk_cost, d_records):
        future = [
            (p, i) for i, p in enumerate(d_records)
            if p['Purchaser_Date'] >= project_date and p['remaining_appetite'] >= chunk_cost
        ]
        if not future:
            return None
        # Sort by earliest Purchaser_Date
        future.sort(key=lambda x: x[0]['Purchaser_Date'])
        return future[0]

    # Helper to re-sort demand back to original logic (appetite desc, CA-first):
    def resort_demand_to_original(d_records):
        d_records.sort(key=lambda x: (-x['approximated_appetite'], x['State Priority']))
        return d_records

    # Group supply by Project Name in the same order they appear
    grouped_supply = supply_df.groupby('Project Name', sort=False)

    # ---------------------------------------------------------------------
    # Allocation loop
    # ---------------------------------------------------------------------
    for project_name, group in grouped_supply:
        if group['Fully Allocated'].all():
            continue

        project_ok = True
        project_chunks = group.to_dict('records')

        # We'll store allocations temporarily, to confirm if entire project can be allocated
        temp_allocs = []
        temp_financed = []

        # Make a local copy of the demand + bank, so if we fail we can roll back
        local_demand = [dict(d) for d in demand_records]
        local_bank = bank_available

        # Sort chunks internally by (priority asc, chunk cost desc)
        sorted_chunks = sorted(
            project_chunks,
            key=lambda x: (x['Priority'], -x['Est Cash Amount'])
        )

        # For each chunk
        for chunk in sorted_chunks:
            if chunk.get('Fully Allocated'):
                continue

            chunk_cost = chunk['Est Cash Amount']
            p_name  = chunk['Project Name']
            p_cat   = chunk.get('Project Category','')
            p_st    = chunk.get('Project State','')
            p_gdf   = chunk['GDF Profit']
            p_id    = chunk['Project ID']
            p_pri   = chunk['Priority']
            p_date  = chunk['Project_date']

            # Default record with "false" result
            allocation_record = {
                "Chunk Est Amount": chunk_cost,  # store the full chunk cost
                "Purchaser Name": None,
                "Purchaser State": None,
                "Purchaser Date": None,
                "Purchaser Category": None,

                "Project Name": p_name,
                "Project Date": p_date,
                "Project Category": p_cat,
                "Project State": p_st,
                "GDF Profit": p_gdf,
                "Priority": p_pri,

                "Allocated Amount": 0.0,
                "Financed Amount": 0.0,
                "Remaining Appetite After": None,
                "Original Appetite": None,

                "Project ID": p_id,
                "Result": False
            }

            # 1) Attempt direct
            direct = find_direct_purchaser(p_date, chunk_cost, local_demand)
            if direct:
                pur, pur_idx = direct
                pur['remaining_appetite'] -= chunk_cost

                allocation_record.update({
                    "Purchaser Name": pur['Purchaser Name'],
                    "Purchaser State": pur.get('Purchaser State',''),
                    "Purchaser Date": pur['Purchaser_Date'],
                    "Purchaser Category": pur.get('Purchaser Category',''),
                    "Allocated Amount": chunk_cost,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": pur['remaining_appetite'],
                    "Original Appetite": pur['approximated_appetite'],
                    "Result": True
                })
                temp_allocs.append(allocation_record)
                continue

            # 2) No direct buyer => Attempt Bank bridging
            if local_bank >= chunk_cost and bank_credit_limit > 0:
                future = find_future_purchaser_earliest_date(p_date, chunk_cost, local_demand)
                if future:
                    f_pur, f_idx = future

                    # Bank "entry"
                    local_bank -= chunk_cost
                    bank_in = allocation_record.copy()
                    bank_in.update({
                        "Purchaser Name": "Bank Credit",
                        "Purchaser State": "N/A",
                        "Purchaser Date": p_date,
                        "Purchaser Category": "N/A",

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": chunk_cost,
                        "Remaining Appetite After": 0,
                        "Original Appetite": 0,
                        "Result": True
                    })
                    temp_allocs.append(bank_in)

                    # Bank "exit" to earliest-date future purchaser
                    f_pur['remaining_appetite'] -= chunk_cost
                    exit_date = f_pur['Purchaser_Date']
                    local_bank += chunk_cost

                    bank_out = allocation_record.copy()
                    bank_out.update({
                        "Purchaser Name": f_pur['Purchaser Name'],
                        "Purchaser State": f_pur.get('Purchaser State',''),
                        "Purchaser Date": exit_date,
                        "Purchaser Category": f_pur.get('Purchaser Category',''),

                        "Allocated Amount": chunk_cost,
                        "Financed Amount": -chunk_cost,  # negative indicates exit from bank
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],
                        "Result": True
                    })
                    temp_allocs.append(bank_out)

                    # Record bridging info
                    temp_financed.append({
                        "Project ID": p_id,
                        "Project Name": p_name,
                        "Chunk Cost": chunk_cost,
                        "Purchase Date": p_date,
                        "Exit Date": exit_date,
                        "Exit Purchaser Name": f_pur['Purchaser Name'],
                        "Remaining Appetite After": f_pur['remaining_appetite'],
                        "Original Appetite": f_pur['approximated_appetite'],
                        "GDF Profit": p_gdf
                    })

                    # Re-sort local_demand
                    local_demand = resort_demand_to_original(local_demand)
                else:
                    # Bridging not possible => fail the project
                    project_ok = False
                    break
            else:
                # No direct buyer & bridging not possible => fail
                project_ok = False
                break

            # If bridging or direct never happened, we still keep one record with Result=False
            # but since we break out if bridging fails, we'll finalize that after the loop.

        # ---------------------------------------------------------------------
        # Decide if the project was fully allocated
        # ---------------------------------------------------------------------
        if project_ok:
            # Mark as allocated
            allocated_projects.add(project_name)

            # Commit local changes
            demand_records = local_demand
            bank_available = local_bank

            # Add to final allocations
            # (We already appended them in the loop)
            pass
            # Mark supply "Fully Allocated"
            supply_df.loc[group.index, 'Fully Allocated'] = True
        else:
            # If the project was not fully allocated,
            # we revert all temp_allocs for that group
            # and add a "false" record for each chunk (so we see them in final output)
            # But those were partially appended. Let's ensure we revert that:
            temp_allocs = []
            for ch in group.to_dict('records'):
                chunk_cost = ch['Est Cash Amount']
                p_gdf = ch['GDF Profit']
                temp_allocs.append({
                    "Chunk Est Amount": chunk_cost,
                    "Purchaser Name": None,
                    "Purchaser State": None,
                    "Purchaser Date": None,
                    "Purchaser Category": None,
                    "Project Name": ch['Project Name'],
                    "Project Date": ch['Project_date'],
                    "Project Category": ch.get('Project Category',''),
                    "Project State": ch.get('Project State',''),
                    "GDF Profit": p_gdf,
                    "Priority": ch['Priority'],
                    "Allocated Amount": 0.0,
                    "Financed Amount": 0.0,
                    "Remaining Appetite After": None,
                    "Original Appetite": None,
                    "Project ID": ch['Project ID'],
                    "Result": False
                })

        allocations.extend(temp_allocs)
        financed_chunks.extend(temp_financed)

    # ---------------------------------------------------------------------
    # Build Allocations DataFrame
    # ---------------------------------------------------------------------
    allocations_df = pd.DataFrame(allocations)
    alloc_cols = [
        "Purchaser Name","Purchaser State","Purchaser Date","Purchaser Category",
        "Project Name","Project Date","Project Category","Project State","GDF Profit",
        "Priority","Chunk Est Amount","Allocated Amount","Financed Amount",
        "Remaining Appetite After","Original Appetite","Project ID","Result"
    ]
    # Keep only the columns that exist
    allocations_df = allocations_df[[c for c in alloc_cols if c in allocations_df.columns]]

    # IMPORTANT: This column now represents the "theoretical" gross profit for *each chunk*
    # regardless of whether Result=True or False.
    # i.e. chunk_est_amount * GDF_Profit
    allocations_df['Chunk Gross Profit'] = (
        allocations_df['Chunk Est Amount'] * allocations_df['GDF Profit']
    )

    # ---------------------------------------------------------------------
    # Build Financing DataFrame
    # ---------------------------------------------------------------------
    financed_df = pd.DataFrame(financed_chunks)
    fin_cols = [
        "Project ID","Project Name","Chunk Cost","Purchase Date","Exit Date",
        "Exit Purchaser Name","Remaining Appetite After","Original Appetite",
        "GDF Profit"
    ]
    for cc in fin_cols:
        if cc not in financed_df.columns:
            financed_df[cc] = None
    financed_df = financed_df[fin_cols]

    # If bridging was used, calculate interest & profit
    if not financed_df.empty:
        financed_df['Purchase Date'] = pd.to_datetime(financed_df['Purchase Date'], errors='coerce')
        financed_df['Exit Date'] = pd.to_datetime(financed_df['Exit Date'], errors='coerce')

        # Months of bank credit
        financed_df['Time of Bank Credit'] = financed_df.apply(
            lambda row: (
                (row['Exit Date'].year - row['Purchase Date'].year) * 12
                + (row['Exit Date'].month - row['Purchase Date'].month)
            ),
            axis=1
        )

        # 1% monthly interest
        financed_df['Cost of Interest'] = financed_df['Chunk Cost'] * 0.01 * financed_df['Time of Bank Credit']

        financed_df['Gross Profit'] = financed_df['Chunk Cost'] * financed_df['GDF Profit']
        financed_df['Net Profit']   = financed_df['Gross Profit'] - financed_df['Cost of Interest']
        financed_df['Net profit %'] = financed_df['Net Profit'] / financed_df['Chunk Cost']
    else:
        financed_df['Time of Bank Credit'] = None
        financed_df['Cost of Interest'] = None
        financed_df['Gross Profit'] = None
        financed_df['Net Profit'] = None
        financed_df['Net profit %'] = None

    # ---------------------------------------------------------------------
    # Summary Metrics
    # ---------------------------------------------------------------------
    # Total allocated (exclude the temporary "Bank Credit" allocations)
    total_alloc = (
        allocations_df['Allocated Amount'].sum()
        - allocations_df.loc[allocations_df['Purchaser Name'] == 'Bank Credit', 'Allocated Amount'].sum()
    )
    financed_used = allocations_df.loc[allocations_df['Financed Amount'] > 0, 'Financed Amount'].sum()
    bank_remaining = bank_available

    # Demand side
    original_total_appetite = demand_df['approximated_appetite'].sum()
    final_demand_records = pd.DataFrame(demand_records)
    remaining_demand = final_demand_records['remaining_appetite'].sum()

    # Supply side
    original_total_project_value = supply_df['Original Est Cash Amount'].sum()
    unallocated_projects = set(supply_df['Project Name'].unique()) - allocated_projects
    unallocated_supply_df = supply_df[supply_df['Project Name'].isin(unallocated_projects)]
    remaining_supply_value = unallocated_supply_df['Est Cash Amount'].sum()

    pct_alloc = 0.0
    if original_total_project_value > 0:
        pct_alloc = (total_alloc / original_total_project_value) * 100

    # Prepare a high-level summary DataFrame
    summary_data = [
        {"Metric": "--- RESULTS SUMMARY ---", "Value": ""},
        {"Metric": "Total Allocated", "Value": f"{total_alloc:,.2f}"},
        {"Metric": "Financed (Bank Used)", "Value": f"{financed_used:,.2f}"},
        {"Metric": "Bank Credit Remaining", "Value": f"{bank_remaining:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "DEMAND SUM", "Value": ""},
        {"Metric": "Original Total Appetite", "Value": f"{original_total_appetite:,.2f}"},
        {"Metric": "Remaining Demand", "Value": f"{remaining_demand:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "% of Allocated Supply", "Value": f"{pct_alloc:.2f}%"},

        {"Metric": "", "Value": ""},
        {"Metric": "SUPPLY SUM", "Value": ""},
        {"Metric": "Original Total Project Value", "Value": f"{original_total_project_value:,.2f}"},
        {"Metric": "Remaining Supply", "Value": f"{remaining_supply_value:,.2f}"},

        {"Metric": "", "Value": ""},
        {"Metric": "--- GROSS PROFIT METRICS ---", "Value": ""},
        {"Metric": "Total Gross Profit (theoretical all)", "Value": ""},  # -3
        {"Metric": "Total Gross Profit (allocated)", "Value": ""},        # -2
        {"Metric": "Total Gross Profit (unallocated)", "Value": ""}       # -1
    ]
    summary_df = pd.DataFrame(summary_data)

    # ---------------------------------------------------------------------
    # Supply Summary
    # ---------------------------------------------------------------------
    # Mark final Results per project (if any chunk got allocated => True)
    # Because a project can have multiple chunks; if at least one chunk == True, let's call project allocated
    # or if you want "fully allocated," you can do `.all()` logic.
    project_results = allocations_df.groupby('Project Name')['Result'].any().reset_index()
    project_results.rename(columns={'Result': 'Result'}, inplace=True)

    supply_after_df = supply_df.merge(project_results, on='Project Name', how='left')
    supply_after_df['Result'] = supply_after_df['Result'].fillna(False)

    # Basic aggregator
    supply_summary = supply_after_df.groupby('Project Name').agg(
        Total_Chunks_Before=('Est Cash Amount', 'count'),
        Total_Project_Value=('Est Cash Amount', 'sum'),
        Latest_Project_Date=('Project_date', 'max'),
        Max_Est_Cash_Amount=('Est Cash Amount', 'max'),
        GDF_Profit=('GDF Profit', 'max'),  # just for reference
        Priority=('Priority', 'max')
    ).reset_index()

    # 1) Theoretical GP = sum of chunk_gross_profit for all chunks (Result=TRUE or FALSE)
    #    We get that from allocations_df, grouping by project name
    gp_theoretical = (
        allocations_df
        .groupby('Project Name')['Chunk Gross Profit']
        .sum()
        .reset_index(name='Gross_Profit_Theoretical')
    )

    # 2) Allocated GP = sum of chunk_gross_profit but only where (Result=True)
    gp_allocated = (
        allocations_df
        .loc[allocations_df['Result'] == True]
        .groupby('Project Name')['Chunk Gross Profit']
        .sum()
        .reset_index(name='Gross_Profit_Allocated')
    )

    # Merge into supply_summary
    supply_summary = supply_summary.merge(gp_theoretical, on='Project Name', how='left')
    supply_summary = supply_summary.merge(gp_allocated, on='Project Name', how='left')

    supply_summary['Gross_Profit_Theoretical'] = supply_summary['Gross_Profit_Theoretical'].fillna(0)
    supply_summary['Gross_Profit_Allocated']   = supply_summary['Gross_Profit_Allocated'].fillna(0)

    # 3) Unallocated portion
    supply_summary['Gross_Profit_Unallocated'] = (
        supply_summary['Gross_Profit_Theoretical'] - supply_summary['Gross_Profit_Allocated']
    )

    # Totals for summary
    total_gross_profit_theoretical = supply_summary['Gross_Profit_Theoretical'].sum()
    total_gross_profit_allocated   = supply_summary['Gross_Profit_Allocated'].sum()
    total_gross_profit_unallocated = supply_summary['Gross_Profit_Unallocated'].sum()

    # Fill these into summary_df
    summary_df.iloc[-3, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_theoretical:,.2f}"
    summary_df.iloc[-2, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_allocated:,.2f}"
    summary_df.iloc[-1, summary_df.columns.get_loc("Value")] = f"{total_gross_profit_unallocated:,.2f}"

    # ---------------------------------------------------------------------
    # Demand After
    # ---------------------------------------------------------------------
    updated_demand_df = final_demand_records.copy()
    updated_demand_df['allocated_amount'] = (
        updated_demand_df['approximated_appetite'] - updated_demand_df['remaining_appetite']
    )
    updated_demand_df["Financing"] = 0.0

    # If bridging was used, track how much each purchaser financed
    if not allocations_df.empty and "Financed Amount" in allocations_df.columns:
        financing_by_purchaser = (
            allocations_df[allocations_df['Financed Amount'] < 0]
            .groupby('Purchaser Name')['Financed Amount'].sum()
            .abs()
            .reset_index(name='Financing')
        )
        updated_demand_df = updated_demand_df.merge(
            financing_by_purchaser, on='Purchaser Name', how='left', suffixes=('', '_calc')
        )
        updated_demand_df['Financing'] = updated_demand_df['Financing_calc'].fillna(0)
        updated_demand_df.drop(columns=['Financing_calc'], inplace=True)

    demand_after_cols = [
        "Purchaser Name",
        "approximated_appetite",
        "allocated_amount",
        "Purchaser State",
        "Purchaser_Date",
        "Financing",
        "remaining_appetite",
        "Purchaser Category",
        "State Priority"
    ]
    for c in demand_after_cols:
        if c not in updated_demand_df.columns:
            updated_demand_df[c] = None

    updated_demand_df = updated_demand_df[demand_after_cols]

    return (
        allocations_df,      # 1) Detailed chunk-by-chunk allocations (with full "Chunk Est Amount")
        financed_df,         # 2) Bank bridging details
        summary_df,          # 3) High-level summary
        supply_summary,      # 4) Per-project supply summary (now chunk-based theoretical GP)
        supply_after_df,     # 5) Supply with final project 'Result'
        updated_demand_df    # 6) Demand after final allocations
    )

# -------------------------------------------------------------------------
# 4. Main Execution
# -------------------------------------------------------------------------
def main():
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    (
        allocations_df,
        financed_df,
        summary_df,
        supply_summary,
        updated_supply,
        updated_demand_df
    ) = allocate_chunks_no_partial(supply_df, demand_df, bank_credit_limit=20_000_000)

    # Print the summary to console
    print(summary_df.to_string(index=False))

    # Save to Excel in multiple tabs
    with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
        allocations_df.to_excel(writer, sheet_name="Allocations", index=False)
        financed_df.to_excel(writer, sheet_name="Financing Chunks", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)
        updated_supply.to_excel(writer, sheet_name="SupplyAfter", index=False)
        updated_demand_df.to_excel(writer, sheet_name="DemandAfter", index=False)

    print("\nResults saved to 'allocation_results.xlsx' with:")
    print(" - Allocations (now with 'Chunk Est Amount' and 'Chunk Gross Profit' for TRUE or FALSE)")
    print(" - Financing Chunks (details of bridging, interest & profit columns)")
    print(" - Summary (result metrics including chunk-based total GP)")
    print(" - Supply Summary (per-project chunk-based GP: theoretical vs allocated vs unallocated)")
    print(" - SupplyAfter (indicates overall project success/failure in 'Result')")
    print(" - DemandAfter (final states of demand)")

if __name__ == "__main__":
    main()


                              Metric          Value
             --- RESULTS SUMMARY ---               
                     Total Allocated  66,573,981.15
                Financed (Bank Used)   3,199,292.00
               Bank Credit Remaining  20,000,000.00
                                                   
                          DEMAND SUM               
             Original Total Appetite 115,747,647.00
                    Remaining Demand  49,173,665.85
                                                   
               % of Allocated Supply         39.08%
                                                   
                          SUPPLY SUM               
        Original Total Project Value 170,370,975.38
                    Remaining Supply 103,796,994.23
                                                   
        --- GROSS PROFIT METRICS ---               
Total Gross Profit (theoretical all)  44,134,309.41
      Total Gross Profit (allocated)  17,050,080.09
    Total Gr

**- remaining chunks per size**
**- remaining demand per chunks**

In [7]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

# -------------------------------------------------------------------------
# 1. Load Data from Excel
# -------------------------------------------------------------------------
def load_data(excel_file="allocation_results.xlsx"):
    """
    Loads necessary sheets from the Excel file.

    Parameters:
        excel_file (str): Path to the Excel file.

    Returns:
        dict: A dictionary containing DataFrames for each sheet.
    """
    try:
        xls = pd.ExcelFile(excel_file)
        data = {
            'Allocations': pd.read_excel(xls, 'Allocations'),
            'Financing Chunks': pd.read_excel(xls, 'Financing Chunks'),
            'Summary': pd.read_excel(xls, 'Summary'),
            'Supply Summary': pd.read_excel(xls, 'Supply Summary'),
            'SupplyAfter': pd.read_excel(xls, 'SupplyAfter'),
            'DemandAfter': pd.read_excel(xls, 'DemandAfter')
        }
        return data
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None

# -------------------------------------------------------------------------
# 2. Create Summary Charts
# -------------------------------------------------------------------------
def create_summary_charts(summary_df):
    """
    Creates charts based on the Summary DataFrame.

    Parameters:
        summary_df (DataFrame): The Summary DataFrame.

    Returns:
        list: A list of Plotly Figure objects.
    """
    figures = []

    # Extract relevant metrics
    metrics = summary_df['Metric']
    values = summary_df['Value']

    # Filter out empty metrics and ensure data integrity
    summary_metrics = summary_df[metrics.str.strip() != ''].copy()

    # ----------- Financial Summary Bar Chart -----------
    financial_metrics = summary_metrics[
        summary_metrics['Metric'].isin([
            "Total Allocated",
            "Financed (Bank Used)",
            "Bank Credit Remaining"
        ])
    ].copy()

    # Convert 'Value' to numeric for plotting
    financial_metrics['Numeric Value'] = financial_metrics['Value'].replace('[\$,]', '', regex=True).astype(float)

    fig_financial = px.bar(
        financial_metrics,
        x='Metric',
        y='Numeric Value',
        title='Financial Summary',
        text='Numeric Value',
        labels={'Numeric Value': 'Amount ($)', 'Metric': ''}
    )
    fig_financial.update_traces(texttemplate='$%{text:,.2f}', textposition='outside', marker_color='indianred')
    fig_financial.update_layout(
        yaxis=dict(title='Amount ($)', automargin=True),
        xaxis_title="",
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        height=600,  # Consistent height
        margin=dict(t=100, b=150),  # Adjust top and bottom margins
        template='plotly_white'
    )

    figures.append(fig_financial)

    # ----------- Demand Appetite Original vs Remaining Pie Chart -----------
    # Extract 'Original Total Appetite' and 'Remaining Demand'
    demand_original = summary_metrics[summary_metrics['Metric'] == "Original Total Appetite"].copy()
    demand_remaining = summary_metrics[summary_metrics['Metric'] == "Remaining Demand"].copy()

    if not demand_original.empty and not demand_remaining.empty:
        original_demand = demand_original['Value'].str.replace('[\$,]', '', regex=True).astype(float).sum()
        remaining_demand = demand_remaining['Value'].str.replace('[\$,]', '', regex=True).astype(float).sum()

        if original_demand > 0:
            remaining_percentage = (remaining_demand / original_demand) * 100
            used_percentage = 100 - remaining_percentage

            demand_comparison = pd.DataFrame({
                'State': ['Remaining', 'Used'],
                'Percentage': [remaining_percentage, used_percentage],
                'Amount': [remaining_demand, original_demand - remaining_demand]
            })

            fig_demand = px.pie(
                demand_comparison,
                names='State',
                values='Percentage',
                title=f'Demand Appetite: Original ${original_demand:,.2f}',
                hole=0.3,
                labels={'Percentage': 'Percentage (%)', 'State': ''},
                custom_data=['Amount']
            )
            fig_demand.update_traces(
                textinfo='percent+label',
                textposition='inside',
                hovertemplate='<b>%{label}</b><br>Amount: $%{customdata[0]:,.2f}<br>Percentage: %{percent}'
            )
            fig_demand.update_layout(
                height=600,  # Consistent height
                margin=dict(t=100),  # Adjust top margin
                template='plotly_white'
            )
            figures.append(fig_demand)
        else:
            print("Original Total Appetite is zero or missing. Skipping Demand Appetite Pie Chart.")
    else:
        print("Demand metrics missing. Skipping Demand Appetite Pie Chart.")

    # ----------- Supply Overview Original vs Remaining Pie Chart -----------
    # Extract 'Original Total Project Value' and 'Remaining Supply'
    supply_original = summary_metrics[summary_metrics['Metric'] == "Original Total Project Value"].copy()
    supply_remaining = summary_metrics[summary_metrics['Metric'] == "Remaining Supply"].copy()

    if not supply_original.empty and not supply_remaining.empty:
        original_supply = supply_original['Value'].str.replace('[\$,]', '', regex=True).astype(float).sum()
        remaining_supply = supply_remaining['Value'].str.replace('[\$,]', '', regex=True).astype(float).sum()

        if original_supply > 0:
            remaining_percentage = (remaining_supply / original_supply) * 100
            used_percentage = 100 - remaining_percentage

            supply_comparison = pd.DataFrame({
                'State': ['Remaining', 'Used'],
                'Percentage': [remaining_percentage, used_percentage],
                'Amount': [remaining_supply, original_supply - remaining_supply]
            })

            fig_supply = px.pie(
                supply_comparison,
                names='State',
                values='Percentage',
                title=f'Supply Overview: Original ${original_supply:,.2f}',
                hole=0.3,
                labels={'Percentage': 'Percentage (%)', 'State': ''},
                custom_data=['Amount']
            )
            fig_supply.update_traces(
                textinfo='percent+label',
                textposition='inside',
                hovertemplate='<b>%{label}</b><br>Amount: $%{customdata[0]:,.2f}<br>Percentage: %{percent}'
            )
            fig_supply.update_layout(
                height=600,  # Consistent height
                margin=dict(t=100),  # Adjust top margin
                template='plotly_white'
            )
            figures.append(fig_supply)
        else:
            print("Original Total Project Value is zero or missing. Skipping Supply Overview Pie Chart.")
    else:
        print("Supply metrics missing. Skipping Supply Overview Pie Chart.")

    # ----------- Percentage of Allocated Supply Gauge -----------
    pct_alloc = summary_metrics[summary_metrics['Metric'] == '% of Allocated Supply']['Value'].values
    if len(pct_alloc) > 0:
        pct_value_str = pct_alloc[0].strip('%')
        try:
            pct_value = float(pct_value_str)
            fig_pct = go.Figure(go.Indicator(
                mode = "gauge+number",
                value = pct_value,
                title = {'text': "% of Allocated Supply"},
                gauge = {
                    'axis': {'range': [0, 100]},
                    'bar': {'color': "darkblue"},
                    'steps' : [
                        {'range': [0, 50], 'color': "lightgray"},
                        {'range': [50, 100], 'color': "gray"}],
                    'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 90}}))
            fig_pct.update_layout(
                height=600,  # Consistent height
                margin=dict(t=100),  # Adjust top margin
                template='plotly_white'
            )
            figures.append(fig_pct)
        except ValueError:
            print("Invalid '% of Allocated Supply' value. Skipping Gauge Chart.")
    else:
        print("'% of Allocated Supply' metric missing. Skipping Gauge Chart.")

    # ----------- Gross Profit Metrics Bar Chart -----------
    gross_profit_metrics = summary_metrics[summary_metrics['Metric'].isin([
        "Total Gross Profit (all projects)",
        "Total Gross Profit Allocated",
        "Total Gross Profit Unallocated"
    ])].copy()

    if not gross_profit_metrics.empty:
        # Convert 'Value' to numeric
        gross_profit_metrics['Numeric Value'] = gross_profit_metrics['Value'].replace('[\$,]', '', regex=True).astype(float)

        fig_gross = px.bar(
            gross_profit_metrics,
            x='Metric',
            y='Numeric Value',
            title='Gross Profit Metrics',
            text='Numeric Value',
            labels={'Numeric Value': 'Amount ($)', 'Metric': ''}
        )
        fig_gross.update_traces(texttemplate='$%{text:,.2f}', textposition='outside', marker_color='seagreen')
        fig_gross.update_layout(
            yaxis=dict(title='Amount ($)', automargin=True),
            xaxis_title="",
            uniformtext_minsize=8,
            uniformtext_mode='hide',
            height=600,  # Consistent height
            margin=dict(t=100, b=150),  # Adjust top and bottom margins
            template='plotly_white'
        )
        figures.append(fig_gross)
    else:
        print("Gross Profit metrics missing. Skipping Gross Profit Metrics Bar Chart.")

    return figures

# -------------------------------------------------------------------------
# 3. Create Supply After Charts
# -------------------------------------------------------------------------
def create_supply_after_charts(supply_after_df):
    """
    Creates charts based on the SupplyAfter DataFrame.

    Parameters:
        supply_after_df (DataFrame): The SupplyAfter DataFrame.

    Returns:
        list: A list of Plotly Figure objects.
    """
    figures = []

    # ----------- Project Allocation Status Pie Chart -----------
    allocation_status = supply_after_df['Result'].value_counts().reset_index()
    allocation_status.columns = ['Result', 'Count']
    allocation_status['Result'] = allocation_status['Result'].map({True: 'Allocated', False: 'Not Allocated'})

    fig_alloc_status = px.pie(
        allocation_status,
        names='Result',
        values='Count',
        title='Project Allocation Status',
        hole=0.3,
        labels={'Count': 'Number of Projects', 'Result': ''},
        custom_data=['Count']
    )
    fig_alloc_status.update_traces(
        textinfo='percent+label',
        textposition='inside',
        hovertemplate='<b>%{label}</b><br>Count: %{customdata[0]}<br>Percentage: %{percent}'
    )
    fig_alloc_status.update_layout(
        height=600,  # Consistent height
        margin=dict(t=100),  # Adjust top margin
        template='plotly_white'
    )
    figures.append(fig_alloc_status)

    # ----------- Project Category Distribution Bar Chart -----------
    if 'Project Category' in supply_after_df.columns:
        project_categories = supply_after_df['Project Category'].value_counts().reset_index()
        project_categories.columns = ['Project Category', 'Count']

        fig_proj_cat = px.bar(
            project_categories,
            x='Project Category',
            y='Count',
            title='Project Category Distribution',
            labels={'Count': 'Number of Projects', 'Project Category': 'Category'},
            text='Count'
        )
        fig_proj_cat.update_traces(
            texttemplate='%{text}',
            textposition='outside',
            marker_color='mediumseagreen'
        )
        fig_proj_cat.update_layout(
            yaxis=dict(title='Number of Projects', automargin=True),
            xaxis_title="",
            uniformtext_minsize=8,
            uniformtext_mode='hide',
            height=600,  # Consistent height
            margin=dict(t=100, b=150),  # Adjust top and bottom margins
            template='plotly_white'
        )

        # Rotate x-axis labels to prevent overlap
        fig_proj_cat.update_xaxes(tickangle=-45)

        figures.append(fig_proj_cat)

    # ----------- GDF Profit Cumulative Timeline -----------
    if 'GDF Profit' in supply_after_df.columns and 'Project_date' in supply_after_df.columns:
        # Convert 'Project_date' to datetime if not already
        supply_after_df['Project_date'] = pd.to_datetime(supply_after_df['Project_date'], errors='coerce')

        # Sort by Project_date
        sorted_supply = supply_after_df.sort_values('Project_date').copy()

        # Calculate Gross Profit per project
        # Assuming 'GDF Profit' is a decimal (e.g., 0.1528 for 15.28%) and 'Original Est Cash Amount' exists
        # Adjust if 'Original Est Cash Amount' is named differently
        if 'Original Est Cash Amount' in sorted_supply.columns:
            sorted_supply['Gross_Profit'] = sorted_supply['Original Est Cash Amount'] * sorted_supply['GDF Profit']
        else:
            sorted_supply['Gross_Profit'] = sorted_supply['Est Cash Amount'] * sorted_supply['GDF Profit']

        # Calculate cumulative gross profit
        sorted_supply['Cumulative_Gross_Profit'] = sorted_supply['Gross_Profit'].cumsum()

        # Plot cumulative gross profit over time
        fig_gdf_timeline = px.line(
            sorted_supply,
            x='Project_date',
            y='Cumulative_Gross_Profit',
            title='Cumulative Gross Profit Over Time',
            labels={'Project_date': 'Project Date', 'Cumulative_Gross_Profit': 'Cumulative Gross Profit ($)'}
        )
        fig_gdf_timeline.update_traces(line=dict(color='firebrick', width=4))
        fig_gdf_timeline.update_layout(
            yaxis=dict(title='Cumulative Gross Profit ($)', automargin=True),
            xaxis_title="",
            height=600,  # Consistent height
            margin=dict(t=100, b=150),  # Adjust top and bottom margins
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=1, label="1y", step="year", stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(visible=True),
                type="date",
                automargin=True
            ),
            template='plotly_white'
        )
        figures.append(fig_gdf_timeline)

    return figures

# -------------------------------------------------------------------------
# 4. Create Demand After Charts
# -------------------------------------------------------------------------
def create_demand_after_charts(demand_after_df):
    """
    Creates charts based on the DemandAfter DataFrame.

    Parameters:
        demand_after_df (DataFrame): The DemandAfter DataFrame.

    Returns:
        list: A list of Plotly Figure objects.
    """
    figures = []

    # ----------- Remaining Appetite by Purchaser Category Bar Chart -----------
    if 'Purchaser Category' in demand_after_df.columns:
        remaining_appetite = demand_after_df.groupby('Purchaser Category')['remaining_appetite'].sum().reset_index()
        fig_remaining_appetite = px.bar(
            remaining_appetite,
            x='Purchaser Category',
            y='remaining_appetite',
            title='Remaining Appetite by Purchaser Category',
            labels={'remaining_appetite': 'Remaining Appetite ($)', 'Purchaser Category': 'Category'},
            text='remaining_appetite'
        )
        fig_remaining_appetite.update_traces(
            texttemplate='$%{text:,.0f}',
            textposition='outside',
            marker_color='royalblue'
        )
        fig_remaining_appetite.update_layout(
            yaxis=dict(title='Remaining Appetite ($)', automargin=True),
            xaxis_title="",
            uniformtext_minsize=8,
            uniformtext_mode='hide',
            height=600,  # Consistent height
            margin=dict(t=100, b=150),  # Adjust top and bottom margins
            template='plotly_white'
        )

        # Rotate x-axis labels to prevent overlap
        fig_remaining_appetite.update_xaxes(tickangle=-45)

        figures.append(fig_remaining_appetite)

    # ----------- Financing Used per Purchaser Bar Chart -----------
    if 'Financing' in demand_after_df.columns:
        financing = demand_after_df.groupby('Purchaser Name')['Financing'].sum().reset_index()
        financing = financing[financing['Financing'] > 0]  # Only show purchasers who used financing
        if not financing.empty:
            fig_financing = px.bar(
                financing,
                x='Purchaser Name',
                y='Financing',
                title='Financing Used by Purchaser',
                labels={'Financing': 'Financing Used ($)', 'Purchaser Name': 'Purchaser'},
                text='Financing'
            )
            fig_financing.update_traces(
                texttemplate='$%{text:,.0f}',
                textposition='outside',
                marker_color='steelblue'
            )
            fig_financing.update_layout(
                yaxis=dict(title='Financing Used ($)', automargin=True),
                xaxis_title="",
                uniformtext_minsize=8,
                uniformtext_mode='hide',
                height=600,  # Consistent height
                margin=dict(t=100, b=150),  # Adjust top and bottom margins
                template='plotly_white'
            )

            # Rotate x-axis labels to prevent overlap
            fig_financing.update_xaxes(tickangle=-45)

            figures.append(fig_financing)

    # ----------- Remaining Appetite Distribution Histogram -----------
    if 'remaining_appetite' in demand_after_df.columns:
        fig_rem_app_dist = px.histogram(
            demand_after_df,
            x='remaining_appetite',
            nbins=20,
            title='Remaining Appetite Distribution',
            labels={'remaining_appetite': 'Remaining Appetite ($)', 'count': 'Number of Purchasers'},
            opacity=0.75,
            color_discrete_sequence=['lightseagreen']
        )
        fig_rem_app_dist.update_layout(
            yaxis=dict(title='Number of Purchasers', automargin=True),
            xaxis_title="Remaining Appetite ($)",
            height=600,  # Consistent height
            margin=dict(t=100, b=150),  # Adjust top and bottom margins
            template='plotly_white'
        )
        figures.append(fig_rem_app_dist)

    return figures

# -------------------------------------------------------------------------
# 5. Display All Charts in an Interactive Dashboard
# -------------------------------------------------------------------------
def create_dashboard(summary_figs, supply_figs, demand_figs):
    """
    Combines all figures into an interactive dashboard using Plotly's subplots.

    Parameters:
        summary_figs (list): List of Summary Plotly Figures.
        supply_figs (list): List of SupplyAfter Plotly Figures.
        demand_figs (list): List of DemandAfter Plotly Figures.

    Returns:
        Plotly Figure: The combined dashboard figure.
    """
    # Combine all figures
    all_figs = summary_figs + supply_figs + demand_figs

    # Determine the type for each subplot based on the figure
    subplot_types = []
    for f in all_figs:
        if isinstance(f, go.Figure):
            if any(isinstance(trace, go.Indicator) for trace in f.data):
                subplot_types.append('indicator')
            elif any(isinstance(trace, go.Pie) for trace in f.data):
                subplot_types.append('pie')
            elif any(isinstance(trace, go.Scatter) for trace in f.data):
                subplot_types.append('xy')  # Line charts
            else:
                subplot_types.append('xy')  # Default to 'xy' for other types
        else:
            subplot_types.append('xy')  # Default to 'xy' if unsure

    # Define the number of rows and columns based on the total number of figures
    cols = 2  # Two columns for better visibility
    rows = (len(all_figs) + cols - 1) // cols  # Ceiling division

    # Define specs for each subplot to specify the type
    specs = []
    for r in range(rows):
        row_specs = []
        for c in range(cols):
            idx = r * cols + c
            if idx < len(subplot_types):
                if subplot_types[idx] == 'pie':
                    row_specs.append({'type': 'domain'})  # 'domain' type for pie charts
                elif subplot_types[idx] == 'indicator':
                    row_specs.append({'type': 'indicator'})
                else:
                    row_specs.append({'type': 'xy'})  # For 'xy' charts like bar, scatter
            else:
                row_specs.append({'type': 'xy'})  # Default type for empty subplots
        specs.append(row_specs)

    # Extract subplot titles safely
    subplot_titles = []
    for f in all_figs:
        # Access the title from layout.title.text if it exists
        title = ""
        if f.layout and f.layout.title and f.layout.title.text:
            title = f.layout.title.text
        subplot_titles.append(title)

    # Create subplots with specified types
    fig = make_subplots(
        rows=rows, cols=cols,
        specs=specs,
        subplot_titles=subplot_titles,
        horizontal_spacing=0.1,
        vertical_spacing=0.15
    )

    current_row = 1
    current_col = 1

    for f in all_figs:
        for trace in f.data:
            fig.add_trace(trace, row=current_row, col=current_col)
        current_col += 1
        if current_col > cols:
            current_col = 1
            current_row +=1

    # Update layout
    fig.update_layout(
        height=800 * rows,  # Increased overall height for better visibility
        width=1600,  # Increased width
        title_text="Allocation Results Dashboard",
        showlegend=False,
        template='plotly_white'
    )

    return fig

# -------------------------------------------------------------------------
# 6. Save and Show the Dashboard
# -------------------------------------------------------------------------
def save_and_show_dashboard(fig, output_file="allocation_dashboard.html"):
    """
    Saves the dashboard to an HTML file and opens it in the browser.

    Parameters:
        fig (Plotly Figure): The dashboard figure.
        output_file (str): The output HTML file name.
    """
    fig.write_html(output_file)
    print(f"Dashboard saved to {output_file}")

# -------------------------------------------------------------------------
# 7. Main Function
# -------------------------------------------------------------------------
def main():
    # Load Data
    data = load_data()
    if data is None:
        return

    # Create Charts
    summary_figs = create_summary_charts(data['Summary'])
    supply_figs = create_supply_after_charts(data['SupplyAfter'])
    demand_figs = create_demand_after_charts(data['DemandAfter'])

    # Create Dashboard
    dashboard = create_dashboard(summary_figs, supply_figs, demand_figs)

    # Save and Show
    save_and_show_dashboard(dashboard)

# Execute the main function
if __name__ == "__main__":
    main()

Dashboard saved to allocation_dashboard.html


In [ ]:
from google.colab import files

# Upload the file
uploaded = files.upload()

In [59]:
import pandas as pd
import numpy as np

# Load the Excel file and Financing Chunks tab
file_path = '/content/allocation_results.xlsx'
financing_chunks_df = pd.read_excel(file_path, sheet_name='Financing Chunks')

# Ensure date columns are in datetime format
financing_chunks_df['Purchase Date'] = pd.to_datetime(financing_chunks_df['Purchase Date'])
financing_chunks_df['Exit Date'] = pd.to_datetime(financing_chunks_df['Exit Date'])

# Create a matrix of active financing months
months = list(range(1, 13))  # Months from January to December
active_months_matrix = pd.DataFrame(0, index=financing_chunks_df.index, columns=months)

# Populate the matrix with 1s where financing was active
for index, row in financing_chunks_df.iterrows():
    start_month = row['Purchase Date'].month
    exit_month = row['Exit Date'].month

    # Apply the logic to mark active months
    for month in months:
        if start_month <= month < exit_month:
            active_months_matrix.loc[index, month] = 1

# Add the active months matrix to the main DataFrame
financing_chunks_df = pd.concat([financing_chunks_df, active_months_matrix], axis=1)

# Calculate the total financing amounts across active months
financing_amounts_matrix = active_months_matrix.multiply(financing_chunks_df['Chunk Cost'], axis=0)
financing_amounts_matrix['Project Name'] = financing_chunks_df['Project Name']  # Add Project Name column
financing_chunks_df['Total Financing Amount'] = financing_amounts_matrix.drop(columns=['Project Name']).sum(axis=1)

# Calculate the cost of financed projects based on interest rate
interest_rate = 0.01  # 1% monthly interest
interest_cost_matrix = financing_amounts_matrix.drop(columns=['Project Name']) * interest_rate
interest_cost_matrix['Project Name'] = financing_chunks_df['Project Name']  # Add Project Name column
financing_chunks_df['Total Interest Cost'] = interest_cost_matrix.drop(columns=['Project Name']).sum(axis=1)

# Group by Project Name and create matrices for each project (Active Financing and Interest Cost)
project_financing_matrices = {}
project_interest_matrices = {}

for project_name, group in financing_chunks_df.groupby('Project Name'):
    # Select rows for the current project from financing and interest matrices
    financing_matrix = financing_amounts_matrix.loc[group.index].drop(columns=['Project Name'])
    interest_matrix = interest_cost_matrix.loc[group.index].drop(columns=['Project Name'])

    project_financing_matrices[project_name] = financing_matrix
    project_interest_matrices[project_name] = interest_matrix

# Save results to an Excel file
output_file = '/content/financing_matrices_corrected.xlsx'
with pd.ExcelWriter(output_file) as writer:
    # Save the main DataFrame
    financing_chunks_df.to_excel(writer, sheet_name='Financing Data', index=False)

    # Save the Financing Amounts matrix
    financing_amounts_matrix.to_excel(writer, sheet_name='Financing Amounts', index=False)

    # Save the Interest Costs matrix
    interest_cost_matrix.to_excel(writer, sheet_name='Interest Costs', index=False)

    # Save individual project matrices with sanitized sheet names
    for project_name, matrix in project_financing_matrices.items():
        sanitized_name = f'Fin_{project_name[:25]}'.replace(' ', '_')  # Truncate and replace spaces
        matrix.to_excel(writer, sheet_name=sanitized_name, index=False)

    for project_name, matrix in project_interest_matrices.items():
        sanitized_name = f'Int_{project_name[:25]}'.replace(' ', '_')  # Truncate and replace spaces
        matrix.to_excel(writer, sheet_name=sanitized_name, index=False)

print(f"Results saved to {output_file}")

Results saved to /content/financing_matrices_corrected.xlsx


**adding totals to financing tabs**

In [60]:
import pandas as pd
import numpy as np

# Load the Excel file and Financing Chunks tab
file_path = '/content/allocation_results.xlsx'
financing_chunks_df = pd.read_excel(file_path, sheet_name='Financing Chunks')

# Ensure date columns are in datetime format
financing_chunks_df['Purchase Date'] = pd.to_datetime(financing_chunks_df['Purchase Date'])
financing_chunks_df['Exit Date'] = pd.to_datetime(financing_chunks_df['Exit Date'])

# Create a matrix of active financing months
months = list(range(1, 13))  # Months from January to December
active_months_matrix = pd.DataFrame(0, index=financing_chunks_df.index, columns=months)

# Populate the matrix with 1s where financing was active
for index, row in financing_chunks_df.iterrows():
    start_month = row['Purchase Date'].month
    exit_month = row['Exit Date'].month

    # Apply the logic to mark active months
    for month in months:
        if start_month <= month < exit_month:
            active_months_matrix.loc[index, month] = 1

# Add the active months matrix to the main DataFrame
financing_chunks_df = pd.concat([financing_chunks_df, active_months_matrix], axis=1)

# Calculate the total financing amounts across active months
financing_amounts_matrix = active_months_matrix.multiply(financing_chunks_df['Chunk Cost'], axis=0)
financing_amounts_matrix['Project Name'] = financing_chunks_df['Project Name']  # Add Project Name column
financing_chunks_df['Total Financing Amount'] = financing_amounts_matrix.drop(columns=['Project Name']).sum(axis=1)

# Calculate the cost of financed projects based on interest rate
interest_rate = 0.01  # 1% monthly interest
interest_cost_matrix = financing_amounts_matrix.drop(columns=['Project Name']) * interest_rate
interest_cost_matrix['Project Name'] = financing_chunks_df['Project Name']  # Add Project Name column
financing_chunks_df['Total Interest Cost'] = interest_cost_matrix.drop(columns=['Project Name']).sum(axis=1)

# Compute the row of totals for all periods and add label
total_financing_row = financing_amounts_matrix.drop(columns=['Project Name']).sum().to_frame().T
total_financing_row['Project Name'] = "TOTALS"
total_financing_row.index = ['Total']

total_interest_row = interest_cost_matrix.drop(columns=['Project Name']).sum().to_frame().T
total_interest_row['Project Name'] = "TOTALS"
total_interest_row.index = ['Total']

# Group by Project Name and create matrices for each project (Active Financing and Interest Cost)
project_financing_matrices = {}
project_interest_matrices = {}

for project_name, group in financing_chunks_df.groupby('Project Name'):
    # Select rows for the current project from financing and interest matrices
    financing_matrix = financing_amounts_matrix.loc[group.index].drop(columns=['Project Name'])
    interest_matrix = interest_cost_matrix.loc[group.index].drop(columns=['Project Name'])

    # Create TOTALS row for the project
    project_total_financing = financing_matrix.sum().to_frame().T
    project_total_financing['Project Name'] = f"TOTALS - {project_name}"

    project_total_interest = interest_matrix.sum().to_frame().T
    project_total_interest['Project Name'] = f"TOTALS - {project_name}"

    # Append total row
    financing_matrix = pd.concat([financing_matrix, project_total_financing], ignore_index=True)
    interest_matrix = pd.concat([interest_matrix, project_total_interest], ignore_index=True)

    financing_matrix.index = list(group.index) + ['Total']
    interest_matrix.index = list(group.index) + ['Total']

    project_financing_matrices[project_name] = financing_matrix
    project_interest_matrices[project_name] = interest_matrix

# Save results to an Excel file
output_file = '/content/financing_matrices_corrected.xlsx'
with pd.ExcelWriter(output_file) as writer:
    # Save the main DataFrame
    financing_chunks_df.to_excel(writer, sheet_name='Financing Data', index=False)

    # Save the Financing Amounts matrix with totals
    financing_amounts_matrix = pd.concat([financing_amounts_matrix, total_financing_row], ignore_index=True)
    financing_amounts_matrix.to_excel(writer, sheet_name='Financing Amounts', index=False)

    # Save the Interest Costs matrix with totals
    interest_cost_matrix = pd.concat([interest_cost_matrix, total_interest_row], ignore_index=True)
    interest_cost_matrix.to_excel(writer, sheet_name='Interest Costs', index=False)

    # Save individual project matrices with sanitized sheet names
    for project_name, matrix in project_financing_matrices.items():
        sanitized_name = f'Fin_{project_name[:25]}'.replace(' ', '_')  # Truncate and replace spaces
        matrix.to_excel(writer, sheet_name=sanitized_name, index=False)

    for project_name, matrix in project_interest_matrices.items():
        sanitized_name = f'Int_{project_name[:25]}'.replace(' ', '_')  # Truncate and replace spaces
        matrix.to_excel(writer, sheet_name=sanitized_name, index=False)

print(f"Results saved to {output_file}")



Results saved to /content/financing_matrices_corrected.xlsx


In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

# -------------------------------------------------------------------------
# 1. Load and Process Data from Excel
# -------------------------------------------------------------------------
def load_data(file_path):
    """
    Loads and processes the Financing Chunks data from the Excel file.

    Parameters:
        file_path (str): Path to the Excel file.

    Returns:
        dict: A dictionary containing DataFrames for each relevant sheet.
    """
    try:
        # Load the Excel file
        xls = pd.ExcelFile(file_path)
        print(f"Available sheets: {xls.sheet_names}")

        # Read the necessary sheets
        financing_data = pd.read_excel(xls, 'Financing Data')
        financing_amounts = pd.read_excel(xls, 'Financing Amounts')
        interest_costs = pd.read_excel(xls, 'Interest Costs')

        # Display the columns of Financing Data for verification
        print("\nColumns in 'Financing Data' sheet:")
        print(financing_data.columns.tolist())

        # Verify that required columns exist
        required_columns = ['Project Name', 'Total Financing Amount', 'Total Interest Cost']
        for col in required_columns:
            if col not in financing_data.columns:
                raise KeyError(f"'{col}' column is missing in 'Financing Data' sheet.")

        return {
            'Financing Data': financing_data,
            'Financing Amounts': financing_amounts,
            'Interest Costs': interest_costs
        }
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None

# -------------------------------------------------------------------------
# 2. Create Plotly Charts
# -------------------------------------------------------------------------
def create_charts(data):
    """
    Creates a series of Plotly charts based on the processed data.

    Parameters:
        data (dict): Dictionary containing DataFrames for each sheet.

    Returns:
        list: A list of Plotly Figure objects.
    """
    figures = []

    # Extract DataFrames
    financing_data = data['Financing Data']
    financing_amounts = data['Financing Amounts']
    interest_costs = data['Interest Costs']

    # ----------- Total Financing Amount per Project -----------
    fig_total_financing = px.bar(
        financing_data.sort_values('Total Financing Amount', ascending=False),
        x='Project Name',
        y='Total Financing Amount',
        title='Total Financing Amount per Project',
        labels={'Total Financing Amount': 'Total Financing ($)', 'Project Name': 'Project'},
        text='Total Financing Amount'
    )
    fig_total_financing.update_traces(
        texttemplate='$%{text:,.2f}',
        textposition='outside',
        marker_color='indianred'
    )
    fig_total_financing.update_layout(
        yaxis=dict(title='Total Financing ($)', automargin=True),
        xaxis_title="",
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        height=600,
        margin=dict(t=100, b=150),
        template='plotly_white'
    )
    figures.append(fig_total_financing)

    # ----------- Total Interest Cost per Project -----------
    fig_total_interest = px.bar(
        financing_data.sort_values('Total Interest Cost', ascending=False),
        x='Project Name',
        y='Total Interest Cost',
        title='Total Interest Cost per Project',
        labels={'Total Interest Cost': 'Total Interest Cost ($)', 'Project Name': 'Project'},
        text='Total Interest Cost'
    )
    fig_total_interest.update_traces(
        texttemplate='$%{text:,.2f}',
        textposition='outside',
        marker_color='seagreen'
    )
    fig_total_interest.update_layout(
        yaxis=dict(title='Total Interest Cost ($)', automargin=True),
        xaxis_title="",
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        height=600,
        margin=dict(t=100, b=150),
        template='plotly_white'
    )
    figures.append(fig_total_interest)

    # ----------- Financing Distribution Over Months -----------
    # Aggregate financing per month
    financing_per_month = financing_amounts.drop(columns=['Project Name']).sum().reset_index()
    financing_per_month.columns = ['Month', 'Total Financing']
    financing_per_month['Month Name'] = financing_per_month['Month'].apply(lambda x: datetime.date(1900, x, 1).strftime('%B'))

    fig_financing_distribution = px.bar(
        financing_per_month.sort_values('Month'),
        x='Month Name',
        y='Total Financing',
        title='Financing Distribution Over Months',
        labels={'Total Financing': 'Financing Amount ($)', 'Month Name': 'Month'},
        text='Total Financing'
    )
    fig_financing_distribution.update_traces(
        texttemplate='$%{text:,.2f}',
        textposition='outside',
        marker_color='mediumseagreen'
    )
    fig_financing_distribution.update_layout(
        yaxis=dict(title='Financing Amount ($)', automargin=True),
        xaxis_title="",
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        height=600,
        margin=dict(t=100, b=150),
        template='plotly_white'
    )
    figures.append(fig_financing_distribution)

    # ----------- Interest Cost Distribution Over Months -----------
    # Aggregate interest cost per month
    interest_per_month = interest_costs.drop(columns=['Project Name']).sum().reset_index()
    interest_per_month.columns = ['Month', 'Total Interest Cost']
    interest_per_month['Month Name'] = interest_per_month['Month'].apply(lambda x: datetime.date(1900, x, 1).strftime('%B'))

    fig_interest_distribution = px.bar(
        interest_per_month.sort_values('Month'),
        x='Month Name',
        y='Total Interest Cost',
        title='Interest Cost Distribution Over Months',
        labels={'Total Interest Cost': 'Interest Cost ($)', 'Month Name': 'Month'},
        text='Total Interest Cost'
    )
    fig_interest_distribution.update_traces(
        texttemplate='$%{text:,.2f}',
        textposition='outside',
        marker_color='darkblue'
    )
    fig_interest_distribution.update_layout(
        yaxis=dict(title='Interest Cost ($)', automargin=True),
        xaxis_title="",
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        height=600,
        margin=dict(t=100, b=150),
        template='plotly_white'
    )
    figures.append(fig_interest_distribution)

    # ----------- Top 10 Projects by Financing Amount -----------
    top_10_financing = financing_data.nlargest(10, 'Total Financing Amount')
    fig_top10_financing = px.bar(
        top_10_financing,
        x='Project Name',
        y='Total Financing Amount',
        title='Top 10 Projects by Financing Amount',
        labels={'Total Financing Amount': 'Financing Amount ($)', 'Project Name': 'Project'},
        text='Total Financing Amount'
    )
    fig_top10_financing.update_traces(
        texttemplate='$%{text:,.2f}',
        textposition='outside',
        marker_color='lightsalmon'
    )
    fig_top10_financing.update_layout(
        yaxis=dict(title='Financing Amount ($)', automargin=True),
        xaxis_title="",
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        height=600,
        margin=dict(t=100, b=150),
        template='plotly_white'
    )
    figures.append(fig_top10_financing)

    # ----------- Top 10 Projects by Interest Cost -----------
    top_10_interest = financing_data.nlargest(10, 'Total Interest Cost')
    fig_top10_interest = px.bar(
        top_10_interest,
        x='Project Name',
        y='Total Interest Cost',
        title='Top 10 Projects by Interest Cost',
        labels={'Total Interest Cost': 'Interest Cost ($)', 'Project Name': 'Project'},
        text='Total Interest Cost'
    )
    fig_top10_interest.update_traces(
        texttemplate='$%{text:,.2f}',
        textposition='outside',
        marker_color='mediumturquoise'
    )
    fig_top10_interest.update_layout(
        yaxis=dict(title='Interest Cost ($)', automargin=True),
        xaxis_title="",
        uniformtext_minsize=8,
        uniformtext_mode='hide',
        height=600,
        margin=dict(t=100, b=150),
        template='plotly_white'
    )
    figures.append(fig_top10_interest)

    # ----------- Cumulative Financing and Interest Over Time -----------
    # Aggregate financing per month
    cumulative_financing = financing_per_month['Total Financing'].cumsum()
    cumulative_interest = interest_per_month['Total Interest Cost'].cumsum()

    cumulative_df = pd.DataFrame({
        'Month': financing_per_month['Month Name'],
        'Cumulative Financing': cumulative_financing,
        'Cumulative Interest Cost': cumulative_interest
    })

    fig_cumulative = make_subplots(specs=[[{"secondary_y": True}]])

    fig_cumulative.add_trace(
        go.Scatter(
            x=cumulative_df['Month'],
            y=cumulative_df['Cumulative Financing'],
            name='Cumulative Financing',
            line=dict(color='indianred', width=4)
        ),
        secondary_y=False,
    )

    fig_cumulative.add_trace(
        go.Scatter(
            x=cumulative_df['Month'],
            y=cumulative_df['Cumulative Interest Cost'],
            name='Cumulative Interest Cost',
            line=dict(color='seagreen', width=4)
        ),
        secondary_y=True,
    )

    fig_cumulative.update_layout(
        title_text='Cumulative Financing and Interest Cost Over Time',
        template='plotly_white',
        height=600,
        margin=dict(t=100, b=150)
    )

    fig_cumulative.update_xaxes(title_text="Month")
    fig_cumulative.update_yaxes(title_text="Cumulative Financing ($)", secondary_y=False)
    fig_cumulative.update_yaxes(title_text="Cumulative Interest Cost ($)", secondary_y=True)

    figures.append(fig_cumulative)

    # ----------- Overall Financing and Interest Summary -----------
    # Using Indicator Gauges
    total_financing_all = financing_data['Total Financing Amount'].sum()
    total_interest_all = financing_data['Total Interest Cost'].sum()

    fig_overall_summary = make_subplots(rows=1, cols=2, specs=[[{'type': 'indicator'}, {'type': 'indicator'}]])

    fig_overall_summary.add_trace(
        go.Indicator(
            mode = "number+gauge",
            value = total_financing_all,
            title = {'text': "Total Financing Across All Projects"},
            gauge = {
                'axis': {'range': [0, financing_data['Total Financing Amount'].max() * 1.1]},
                'bar': {'color': "indianred"},
                'steps' : [
                    {'range': [0, financing_data['Total Financing Amount'].max() * 0.5], 'color': "lightgray"},
                    {'range': [financing_data['Total Financing Amount'].max() * 0.5, financing_data['Total Financing Amount'].max() * 1.1], 'color': "gray"}],
                'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': financing_data['Total Financing Amount'].max() * 0.9}}
        ),
        row=1, col=1
    )

    fig_overall_summary.add_trace(
        go.Indicator(
            mode = "number+gauge",
            value = total_interest_all,
            title = {'text': "Total Interest Cost Across All Projects"},
            gauge = {
                'axis': {'range': [0, financing_data['Total Interest Cost'].max() * 1.1]},
                'bar': {'color': "seagreen"},
                'steps' : [
                    {'range': [0, financing_data['Total Interest Cost'].max() * 0.5], 'color': "lightgray"},
                    {'range': [financing_data['Total Interest Cost'].max() * 0.5, financing_data['Total Interest Cost'].max() * 1.1], 'color': "gray"}],
                'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': financing_data['Total Interest Cost'].max() * 0.9}}
        ),
        row=1, col=2
    )

    fig_overall_summary.update_layout(
        title_text="Overall Financing and Interest Summary",
        template='plotly_white',
        height=600,
        margin=dict(t=100, b=150)
    )

    figures.append(fig_overall_summary)

    return figures

# -------------------------------------------------------------------------
# 3. Create Interactive Dashboard
# -------------------------------------------------------------------------
def create_dashboard(figures):
    """
    Combines all Plotly figures into an interactive dashboard.

    Parameters:
        figures (list): List of Plotly Figure objects.

    Returns:
        Plotly Figure: Combined dashboard figure.
    """
    num_figs = len(figures)
    cols = 2
    rows = (num_figs + cols - 1) // cols

    # Define subplot types based on figure data
    subplot_specs = []
    subplot_titles = []
    for fig in figures:
        if any(isinstance(trace, go.Indicator) for trace in fig.data):
            subplot_specs.append({'type': 'indicator'})
        elif any(isinstance(trace, go.Scatter) for trace in fig.data):
            subplot_specs.append({'type': 'xy'})
        elif any(isinstance(trace, go.Bar) for trace in fig.data):
            subplot_specs.append({'type': 'xy'})
        else:
            subplot_specs.append({'type': 'xy'})

        # Extract titles
        title = fig.layout.title.text if fig.layout and fig.layout.title and fig.layout.title.text else ""
        subplot_titles.append(title)

    # Create specs for subplots
    specs = []
    for r in range(rows):
        row_specs = []
        for c in range(cols):
            idx = r * cols + c
            if idx < len(subplot_specs):
                row_specs.append(subplot_specs[idx])
            else:
                row_specs.append({'type': 'xy'})
        specs.append(row_specs)

    # Create subplots
    dashboard = make_subplots(
        rows=rows,
        cols=cols,
        specs=specs,
        subplot_titles=subplot_titles,
        horizontal_spacing=0.1,
        vertical_spacing=0.15
    )

    # Add traces to subplots
    current_row = 1
    current_col = 1
    for fig in figures:
        for trace in fig.data:
            dashboard.add_trace(trace, row=current_row, col=current_col)
        current_col +=1
        if current_col > cols:
            current_col =1
            current_row +=1

    # Update layout
    dashboard.update_layout(
        height=600 * rows,
        width=1600,
        title_text="Financing Chunks Analysis Dashboard",
        showlegend=False,
        template='plotly_white'
    )

    return dashboard

# -------------------------------------------------------------------------
# 4. Save and Display Dashboard
# -------------------------------------------------------------------------
def save_dashboard(dashboard, output_file="financing_dashboard.html"):
    """
    Saves the dashboard to an HTML file.

    Parameters:
        dashboard (Plotly Figure): The combined dashboard figure.
        output_file (str): The output HTML file name.
    """
    dashboard.write_html(output_file)
    print(f"Dashboard successfully saved to {output_file}")

# -------------------------------------------------------------------------
# 5. Main Execution Function
# -------------------------------------------------------------------------
def main():
    # File path to the processed Excel file
    file_path = '/content/financing_matrices_corrected.xlsx'  # Update this path if necessary

    # Load and process data
    data = load_data(file_path)
    if data is None:
        return

    # Create Plotly charts
    figures = create_charts(data)

    # Create Dashboard
    dashboard = create_dashboard(figures)

    # Save Dashboard
    save_dashboard(dashboard)

# -------------------------------------------------------------------------
# 6. Execute the Main Function
# -------------------------------------------------------------------------
if __name__ == "__main__":
    main()

Available sheets: ['Financing Data', 'Financing Amounts', 'Interest Costs', 'Fin_420_Studio', 'Fin_ASCO_Sintering_Co', 'Fin_Baltimore', 'Fin_New_Ringgold', 'Int_420_Studio', 'Int_ASCO_Sintering_Co', 'Int_Baltimore', 'Int_New_Ringgold']

Columns in 'Financing Data' sheet:
['Project ID', 'Project Name', 'Chunk Cost', 'Purchase Date', 'Exit Date', 'Exit Purchaser Name', 'Remaining Appetite After', 'Original Appetite', 'GDF Profit', 'Time of Bank Credit', 'Cost of Interest', 'Gross Profit', 'Net Profit', 'Net profit %', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 'Total Financing Amount', 'Total Interest Cost']
Dashboard successfully saved to financing_dashboard.html
